<a href="https://colab.research.google.com/github/JasmineVan/Pyspark_Midterm/blob/main/51800128.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Đồ án giữa kỳ
# Môn: Xử lý dữ liệu lớn
# Học kỳ 1 - Năm học 2022-2023
# Giảng viên: Th.S Nguyễn Thành An
# Học viên: Trịnh Vân Thương - 51800128

# Cài đặt PySpark

In [222]:
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q http://archive.apache.org/dist/spark/spark-3.1.1/spark-3.1.1-bin-hadoop3.2.tgz
!tar xf spark-3.1.1-bin-hadoop3.2.tgz
!pip install -q findspark

In [223]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.1.1-bin-hadoop3.2"

import findspark
findspark.init()

# Yêu cầu

## Spark Context

In [259]:
from pyspark import SparkContext
from pyspark.sql import SQLContext

# sc.stop()
sc = SparkContext("local", "midterm project")

In [227]:
sqlc = SQLContext(sc)

## Đọc dữ liệu data.csv

In [228]:
data = sc.textFile('/content/data.csv') 
print(type(data))

<class 'pyspark.rdd.RDD'>


## Câu 1: Đếm món hàng

In [229]:
'''
Câu 1
'''
# Read data in each line
# Mapping each row with key "1"
# Reduce by key, incresing the value of key when meet duplicated row (the same Member_number, Date)
items = data\
        .flatMap(lambda x: x.split('/n')) \
        .map(lambda x: (
            (x.split(',')[0], x.split(',')[1]), 1)
        )\
        .reduceByKey(lambda x, y: x + y)

# Using temporary array listData to store data of 3 columns: Member_number, Date and key after increasing
# Save to folder /content/counters
items = items.collect()
listData = []
for item in items:
  # print(item)
  item = item[0][0]+', '+item[0][1]+', '+str(item[1])
  listData.append(item)

cau1NewRdd = sc.parallelize(listData)  
cau1NewRdd.saveAsTextFile('/content/counters')

In [230]:
import os

if os.path.exists('/content/counters/part-00000'):
  sqlc.read.csv('/content/counters/part-00000', header=True).show()

+-------------+-----------+---+
|Member_number|       Date|  1|
+-------------+-----------+---+
|         1249| 01/01/2014|  2|
|         1381| 01/01/2014|  2|
|         1440| 01/01/2014|  2|
|         1659| 01/01/2014|  2|
|         1789| 01/01/2014|  2|
|         1922| 01/01/2014|  2|
|         2226| 01/01/2014|  2|
|         2237| 01/01/2014|  2|
|         2351| 01/01/2014|  2|
|         2542| 01/01/2014|  2|
|         2610| 01/01/2014|  3|
|         2709| 01/01/2014|  2|
|         2727| 01/01/2014|  2|
|         2943| 01/01/2014|  2|
|         2974| 01/01/2014|  3|
|         3681| 01/01/2014|  3|
|         3797| 01/01/2014|  2|
|         3942| 01/01/2014|  3|
|         3956| 01/01/2014|  4|
|         4260| 01/01/2014|  2|
+-------------+-----------+---+
only showing top 20 rows



## Câu 2: Giỏ hàng

In [231]:
'''
Câu 2
'''
# Read data in each line
# Mapping each row with key "1"
# Reduce by key: concaternate data in column itemDescription
baskets = data \
        .flatMap(lambda x: x.split('/n')) \
        .map(lambda x: (
            (x.split(',')[0], x.split(',')[1]), x.split(',')[2])
        )\
        .reduceByKey(lambda x, y: x +','+ y)

# Using temporary array listData to store data of 3 columns: Member_number, Date and items
# Save to folder /content/baskets
baskets = baskets.collect()
listData = []

for basket in baskets:
  # print(basket)
  basket = basket[0][0]+';'+basket[0][1]+';'+basket[1]
  listData.append(basket)

cau2NewRdd = sc.parallelize(listData)  
cau2NewRdd.saveAsTextFile('/content/baskets')

In [232]:
import os

if os.path.exists('/content/baskets/part-00000'):
  sqlc.read.csv('/content/baskets/part-00000', header=True, sep=';').show()

+-------------+----------+--------------------+
|Member_number|      Date|     itemDescription|
+-------------+----------+--------------------+
|         1249|01/01/2014| citrus fruit,coffee|
|         1381|01/01/2014|           curd,soda|
|         1440|01/01/2014|other vegetables,...|
|         1659|01/01/2014|specialty chocola...|
|         1789|01/01/2014|hamburger meat,ca...|
|         1922|01/01/2014|tropical fruit,ot...|
|         2226|01/01/2014|sausage,bottled w...|
|         2237|01/01/2014|bottled water,Ins...|
|         2351|01/01/2014|cleaner,shopping ...|
|         2542|01/01/2014|sliced cheese,bot...|
|         2610|01/01/2014|hamburger meat,bo...|
|         2709|01/01/2014|yogurt,frozen veg...|
|         2727|01/01/2014|hamburger meat,fr...|
|         2943|01/01/2014|whole milk,flower...|
|         2974|01/01/2014|berries,whipped/s...|
|         3681|01/01/2014|onions,whipped/so...|
|         3797|01/01/2014|  waffles,whole milk|
|         3942|01/01/2014|other vegetabl

## Câu 3: Tập phổ biến

In [233]:
'''
Câu 3
'''
path = '/content/baskets/part-00000'
df = sqlc.read.csv(path, header=True, sep=';')
from pyspark.sql.functions import split, col, array_distinct, when
# Convert colunm itemDesciption data from List to Array and rename
dfBaskets = df.select(col("Member_number"), col("Date"), split(col("itemDescription"),",").alias("Items")) \
    .drop("name")
# Remove duplicated in column Items
dfBaskets = dfBaskets.withColumn("Items", array_distinct("Items"))

dfBaskets.printSchema()
print(type(dfBaskets))
dfBaskets.show()

root
 |-- Member_number: string (nullable = true)
 |-- Date: string (nullable = true)
 |-- Items: array (nullable = true)
 |    |-- element: string (containsNull = true)

<class 'pyspark.sql.dataframe.DataFrame'>
+-------------+----------+--------------------+
|Member_number|      Date|               Items|
+-------------+----------+--------------------+
|         1249|01/01/2014|[citrus fruit, co...|
|         1381|01/01/2014|        [curd, soda]|
|         1440|01/01/2014|[other vegetables...|
|         1659|01/01/2014|[specialty chocol...|
|         1789|01/01/2014|[hamburger meat, ...|
|         1922|01/01/2014|[tropical fruit, ...|
|         2226|01/01/2014|[sausage, bottled...|
|         2237|01/01/2014|[bottled water, I...|
|         2351|01/01/2014|[cleaner, shoppin...|
|         2542|01/01/2014|[sliced cheese, b...|
|         2610|01/01/2014|[hamburger meat, ...|
|         2709|01/01/2014|[yogurt, frozen v...|
|         2727|01/01/2014|[hamburger meat, ...|
|         2943|01/0

In [234]:
from pyspark.ml.fpm import FPGrowth

fpGrowth = FPGrowth(itemsCol="Items", minSupport=0.01, minConfidence=0.1)

In [235]:
# Create model
model = fpGrowth.fit(dfBaskets)

In [236]:
# Display frequent itemsets.
model.freqItemsets.show()

+--------------------+----+
|               items|freq|
+--------------------+----+
|              [beef]| 508|
|             [sugar]| 265|
|               [oil]| 223|
|         [chocolate]| 353|
|        [white wine]| 175|
|             [candy]| 215|
|  [processed cheese]| 152|
|              [meat]| 252|
|       [frankfurter]| 565|
|           [sausage]| 903|
|           [berries]| 326|
|            [coffee]| 473|
|         [pip fruit]| 734|
|  [hygiene articles]| 205|
|       [salty snack]| 281|
|       [white bread]| 359|
|[specialty chocol...| 239|
|     [domestic eggs]| 555|
|        [rolls/buns]|1646|
|[rolls/buns, othe...| 158|
+--------------------+----+
only showing top 20 rows



In [237]:
# Display generated association rules.
model.associationRules.show()

+------------------+------------+-------------------+------------------+--------------------+
|        antecedent|  consequent|         confidence|              lift|             support|
+------------------+------------+-------------------+------------------+--------------------+
|[other vegetables]|[whole milk]|0.12151067323481117|0.7694304712706219|0.014836596939116486|
|          [yogurt]|[whole milk]|0.12996108949416343|0.8229402378760761|0.011160863463209249|
|      [rolls/buns]|[whole milk]|0.12697448359659783|0.8040284376030019|0.013967787208447505|
|            [soda]|[whole milk]|0.11975223675154852|0.7582956912879478|0.011628684087415625|
+------------------+------------+-------------------+------------------+--------------------+



## Câu 4: Giỏ hàng thành vectors 

In [238]:
'''
Câu 4
'''
# Similar with exercise 2 but does not contain colunm Date
members = data \
        .flatMap(lambda x: x.split('/n')) \
        .map(lambda x: (
            (x.split(',')[0]), x.split(',')[2])
        )\
        .reduceByKey(lambda x, y: x +','+ y)

# Using temporary array listData to store data of 2 columns: Member_number and Items
# Save to folder /content/baskets
members = members.collect()
listData = []

for member in members:
  # print(member)
  member = member[0]+';'+member[1]
  listData.append(member)

cau4NewRdd = sc.parallelize(listData)  
cau4NewRdd.saveAsTextFile('/content/members')

In [239]:
import os

if os.path.exists('/content/members/part-00000'):
  dfMembers = sqlc.read.csv('/content/members/part-00000', header=True, sep=';')
  dfMembers.show()

+-------------+--------------------+
|Member_number|     itemDescription|
+-------------+--------------------+
|         1249|citrus fruit,coff...|
|         1381|curd,soda,coffee,...|
|         1440|other vegetables,...|
|         1659|specialty chocola...|
|         1789|hamburger meat,ca...|
|         1922|tropical fruit,ot...|
|         2226|sausage,bottled w...|
|         2237|bottled water,Ins...|
|         2351|cleaner,shopping ...|
|         2542|sliced cheese,bot...|
|         2610|hamburger meat,bo...|
|         2709|yogurt,frozen veg...|
|         2727|hamburger meat,fr...|
|         2943|whole milk,flower...|
|         2974|berries,whipped/s...|
|         3681|onions,whipped/so...|
|         3797|waffles,whole mil...|
|         3942|other vegetables,...|
|         3956|yogurt,shopping b...|
|         4260|soda,brown bread,...|
+-------------+--------------------+
only showing top 20 rows



In [240]:
# Rename colunm itemDescription to Items
dfMembers = dfMembers.withColumnRenamed('itemDescription', 'Items')
dfMembers.printSchema()

root
 |-- Member_number: string (nullable = true)
 |-- Items: string (nullable = true)



In [241]:
# Iterate through dfMembers
items = []
for i in dfMembers.collect():
  # Split item in the same row
  itemInTheSameRow = i.Items.split(',')
  for j in itemInTheSameRow:
    # If items contains j then skip this item, else append to items
    if j not in items:
      items.append(j)

# reverse=False will sort items in this list by ASC order
items.sort(reverse=False)
print(items)

['Instant food products', 'UHT-milk', 'abrasive cleaner', 'artif. sweetener', 'baby cosmetics', 'bags', 'baking powder', 'bathroom cleaner', 'beef', 'berries', 'beverages', 'bottled beer', 'bottled water', 'brandy', 'brown bread', 'butter', 'butter milk', 'cake bar', 'candles', 'candy', 'canned beer', 'canned fish', 'canned fruit', 'canned vegetables', 'cat food', 'cereals', 'chewing gum', 'chicken', 'chocolate', 'chocolate marshmallow', 'citrus fruit', 'cleaner', 'cling film/bags', 'cocoa drinks', 'coffee', 'condensed milk', 'cooking chocolate', 'cookware', 'cream', 'cream cheese ', 'curd', 'curd cheese', 'decalcifier', 'dental care', 'dessert', 'detergent', 'dish cleaner', 'dishes', 'dog food', 'domestic eggs', 'female sanitary products', 'finished products', 'fish', 'flour', 'flower (seeds)', 'flower soil/fertilizer', 'frankfurter', 'frozen chicken', 'frozen dessert', 'frozen fish', 'frozen fruits', 'frozen meals', 'frozen potato products', 'frozen vegetables', 'fruit/vegetable juic

In [242]:
# Create dictionary dictItems from list items
dictItems = dict.fromkeys(items)

print(dictItems)

{'Instant food products': None, 'UHT-milk': None, 'abrasive cleaner': None, 'artif. sweetener': None, 'baby cosmetics': None, 'bags': None, 'baking powder': None, 'bathroom cleaner': None, 'beef': None, 'berries': None, 'beverages': None, 'bottled beer': None, 'bottled water': None, 'brandy': None, 'brown bread': None, 'butter': None, 'butter milk': None, 'cake bar': None, 'candles': None, 'candy': None, 'canned beer': None, 'canned fish': None, 'canned fruit': None, 'canned vegetables': None, 'cat food': None, 'cereals': None, 'chewing gum': None, 'chicken': None, 'chocolate': None, 'chocolate marshmallow': None, 'citrus fruit': None, 'cleaner': None, 'cling film/bags': None, 'cocoa drinks': None, 'coffee': None, 'condensed milk': None, 'cooking chocolate': None, 'cookware': None, 'cream': None, 'cream cheese ': None, 'curd': None, 'curd cheese': None, 'decalcifier': None, 'dental care': None, 'dessert': None, 'detergent': None, 'dish cleaner': None, 'dishes': None, 'dog food': None, 

In [243]:
# Update value of dictionary dictItems
for key in dictItems:
  # print(key, dictItems[key])
  dictItems[key] = items.index(key) 

print(dictItems)

{'Instant food products': 0, 'UHT-milk': 1, 'abrasive cleaner': 2, 'artif. sweetener': 3, 'baby cosmetics': 4, 'bags': 5, 'baking powder': 6, 'bathroom cleaner': 7, 'beef': 8, 'berries': 9, 'beverages': 10, 'bottled beer': 11, 'bottled water': 12, 'brandy': 13, 'brown bread': 14, 'butter': 15, 'butter milk': 16, 'cake bar': 17, 'candles': 18, 'candy': 19, 'canned beer': 20, 'canned fish': 21, 'canned fruit': 22, 'canned vegetables': 23, 'cat food': 24, 'cereals': 25, 'chewing gum': 26, 'chicken': 27, 'chocolate': 28, 'chocolate marshmallow': 29, 'citrus fruit': 30, 'cleaner': 31, 'cling film/bags': 32, 'cocoa drinks': 33, 'coffee': 34, 'condensed milk': 35, 'cooking chocolate': 36, 'cookware': 37, 'cream': 38, 'cream cheese ': 39, 'curd': 40, 'curd cheese': 41, 'decalcifier': 42, 'dental care': 43, 'dessert': 44, 'detergent': 45, 'dish cleaner': 46, 'dishes': 47, 'dog food': 48, 'domestic eggs': 49, 'female sanitary products': 50, 'finished products': 51, 'fish': 52, 'flour': 53, 'flow

In [244]:
print(dfMembers.first()['Member_number'], dfMembers.first()['Items'])

1249 citrus fruit,coffee,soda,frozen meals,bottled beer


In [245]:
from pyspark.ml.linalg import Vectors

# Function basket2vector
def basket2vector(member, basket, dictItems):
  itemInBasketPosition = []
  isItInDictItems = []
  # Get item in basket
  itemInBasket = basket.split(',')
  # Sort and remove duplicated item in basket
  itemInBasket = list(dict.fromkeys(itemInBasket))
  itemInBasket.sort()
  for item in itemInBasket:
    # Check if this item exists in dictItems
    if item in dictItems.keys():
      # Append position to itemInBasketPosition and mark as found in isItInDictItems 
      itemInBasketPosition.append(dictItems[item])
      isItInDictItems.append(1.0)
    else:
      # Append 0 to itemInBasketPosition and mark as  not found in isItInDictItems 
      itemInBasketPosition.append(0)  
      isItInDictItems.append(0.0)

  # print(member)
  # print(itemInBasketPosition)
  # print(isItInDictItems)

  # Create vector spare
  return Vectors.sparse(member, itemInBasketPosition, isItInDictItems)

In [246]:
print(basket2vector(dfMembers.first()['Member_number'], dfMembers.first()['Items'], dictItems))

(1249,[11,30,34,61,138],[1.0,1.0,1.0,1.0,1.0])


## Câu 5: Giỏ hàng tương tự

In [247]:
'''
Câu 5
'''
from pyspark.ml.feature import MinHashLSH, VectorAssembler
from pyspark.sql.functions import when

mh = MinHashLSH(inputCol="Items", outputCol="Hashes", numHashTables=5)

dfMembers.show()

+-------------+--------------------+
|Member_number|               Items|
+-------------+--------------------+
|         1249|citrus fruit,coff...|
|         1381|curd,soda,coffee,...|
|         1440|other vegetables,...|
|         1659|specialty chocola...|
|         1789|hamburger meat,ca...|
|         1922|tropical fruit,ot...|
|         2226|sausage,bottled w...|
|         2237|bottled water,Ins...|
|         2351|cleaner,shopping ...|
|         2542|sliced cheese,bot...|
|         2610|hamburger meat,bo...|
|         2709|yogurt,frozen veg...|
|         2727|hamburger meat,fr...|
|         2943|whole milk,flower...|
|         2974|berries,whipped/s...|
|         3681|onions,whipped/so...|
|         3797|waffles,whole mil...|
|         3942|other vegetables,...|
|         3956|yogurt,shopping b...|
|         4260|soda,brown bread,...|
+-------------+--------------------+
only showing top 20 rows



In [248]:
# This part is quite hard. this block to re-create dfMembers
if os.path.exists('/content/members/part-00000'):
  dfMembers = sqlc.read.csv('/content/members/part-00000', header=True, sep=';')
  dfMembers = dfMembers.withColumnRenamed('itemDescription', 'Items')
  dfMembers.show()

+-------------+--------------------+
|Member_number|               Items|
+-------------+--------------------+
|         1249|citrus fruit,coff...|
|         1381|curd,soda,coffee,...|
|         1440|other vegetables,...|
|         1659|specialty chocola...|
|         1789|hamburger meat,ca...|
|         1922|tropical fruit,ot...|
|         2226|sausage,bottled w...|
|         2237|bottled water,Ins...|
|         2351|cleaner,shopping ...|
|         2542|sliced cheese,bot...|
|         2610|hamburger meat,bo...|
|         2709|yogurt,frozen veg...|
|         2727|hamburger meat,fr...|
|         2943|whole milk,flower...|
|         2974|berries,whipped/s...|
|         3681|onions,whipped/so...|
|         3797|waffles,whole mil...|
|         3942|other vegetables,...|
|         3956|yogurt,shopping b...|
|         4260|soda,brown bread,...|
+-------------+--------------------+
only showing top 20 rows



In [249]:
# Apply change to column Items with function basket2Vector in exercise 5
dfMembers = dfMembers.rdd.map(lambda x: 
    (x.Member_number, basket2vector(x.Member_number, x.Items, dictItems)) 
    ).toDF(["Member_number","Items"])
dfMembers.show()

+-------------+--------------------+
|Member_number|               Items|
+-------------+--------------------+
|         1249|(1249,[11,30,34,6...|
|         1381|(1381,[1,10,11,28...|
|         1440|(1440,[28,64,102,...|
|         1659|(1659,[12,14,26,3...|
|         1789|(1789,[8,18,30,58...|
|         1922|(1922,[10,12,15,1...|
|         2226|(2226,[9,12,23,40...|
|         2237|(2237,[0,12,27,34...|
|         2351|(2351,[31,49,75,1...|
|         2542|(2542,[12,82,88,9...|
|         2610|(2610,[11,49,68,9...|
|         2709|(2709,[12,30,40,4...|
|         2727|(2727,[11,52,56,6...|
|         2943|(2943,[20,29,33,5...|
|         2974|(2974,[9,12,63,70...|
|         3681|(3681,[47,63,88,9...|
|         3797|(3797,[15,38,64,1...|
|         3942|(3942,[0,21,32,35...|
|         3956|(3956,[0,16,21,28...|
|         4260|(4260,[14,20,27,6...|
+-------------+--------------------+
only showing top 20 rows



In [250]:
# The 1st customer is used for approxNearestNeighbors()
TheFirstCustomer = dfMembers.first()
print(TheFirstCustomer.Items)

(1249,[11,30,34,61,138],[1.0,1.0,1.0,1.0,1.0])


In [251]:
model = mh.fit(dfMembers)
model.transform(dfMembers).show()

+-------------+--------------------+--------------------+
|Member_number|               Items|              Hashes|
+-------------+--------------------+--------------------+
|         1249|(1249,[11,30,34,6...|[[2.99438855E8], ...|
|         1381|(1381,[1,10,11,28...|[[2.22517691E8], ...|
|         1440|(1440,[28,64,102,...|[[2.8606765E7], [...|
|         1659|(1659,[12,14,26,3...|[[1.83253101E8], ...|
|         1789|(1789,[8,18,30,58...|[[1.63620806E8], ...|
|         1922|(1922,[10,12,15,1...|[[2.8606765E7], [...|
|         2226|(2226,[9,12,23,40...|[[2.8606765E7], [...|
|         2237|(2237,[0,12,27,34...|[[4.7435052E7], [...|
|         2351|(2351,[31,49,75,1...|[[4.36060912E8], ...|
|         2542|(2542,[12,82,88,9...|[[1.44792519E8], ...|
|         2610|(2610,[11,49,68,9...|[[2.22517691E8], ...|
|         2709|(2709,[12,30,40,4...|[[3.1907115E8], [...|
|         2727|(2727,[11,52,56,6...|[[2.99438855E8], ...|
|         2943|(2943,[20,29,33,5...|[[2.8606765E7], [...|
|         2974

In [252]:
# Compute the locality sensitive hashes for the input rows, then perform approximate
# similarity join.
# We could avoid computing hashes by passing in the already-transformed dataset, e.g.
# `model.approxSimilarityJoin(transformedA, transformedB, 0.6)`
print("Approximately joining dfA and dfB on distance smaller than 0.3:")
model.approxSimilarityJoin(dfMembers, dfMembers, 0.3, distCol="JaccardDistance")\
    .select(col("datasetA.Member_number").alias("Member_number_A"),
            col("datasetB.Member_number").alias("Member_number_B"),
            col("JaccardDistance")).show()

Approximately joining dfA and dfB on distance smaller than 0.3:
+---------------+---------------+------------------+
|Member_number_A|Member_number_B|   JaccardDistance|
+---------------+---------------+------------------+
|           2355|           2355|               0.0|
|           4367|           4367|               0.0|
|           4693|           4693|               0.0|
|           4924|           4924|               0.0|
|           3518|           3518|               0.0|
|           2678|           2678|               0.0|
|           2335|           2335|               0.0|
|           3625|           3625|               0.0|
|           3932|           3932|               0.0|
|           4937|           4937|               0.0|
|           1847|           1847|               0.0|
|           4342|           1056|0.2857142857142857|
|           3319|           3319|               0.0|
|           1596|           1596|               0.0|
|           3063|           3063|  

In [253]:
# Compute the locality sensitive hashes for the input rows, then perform approximate nearest
# neighbor search.
# We could avoid computing hashes by passing in the already-transformed dataset, e.g.
# `model.approxNearestNeighbors(transformedA, key, 2)`
# It may return less than 2 rows when not enough approximate near-neighbor candidates are
# found.
print("Approximately searching dfMembers for 5 nearest neighbors of the first customer:")
model.approxNearestNeighbors(dfMembers, TheFirstCustomer.Items, 5).show()

Approximately searching dfMembers for 5 nearest neighbors of the first customer:
+-------------+--------------------+--------------------+------------------+
|Member_number|               Items|              Hashes|           distCol|
+-------------+--------------------+--------------------+------------------+
|         1249|(1249,[11,30,34,6...|[[2.99438855E8], ...|               0.0|
|         1321|(1321,[11,30,138]...|[[2.99438855E8], ...|               0.4|
|         1263|(1263,[11,30,61,1...|[[2.99438855E8], ...|               0.5|
|         1794|(1794,[11,30,138,...|[[2.99438855E8], ...|0.5714285714285714|
|         4327|(4327,[30,34,63,7...|[[2.02885396E8], ...|0.5714285714285714|
+-------------+--------------------+--------------------+------------------+



## Câu 6: Phân cụm người dùng theo giỏ hàng

In [254]:
'''
Câu 6
'''
dfMembers.show()
dfMembers.printSchema()

+-------------+--------------------+
|Member_number|               Items|
+-------------+--------------------+
|         1249|(1249,[11,30,34,6...|
|         1381|(1381,[1,10,11,28...|
|         1440|(1440,[28,64,102,...|
|         1659|(1659,[12,14,26,3...|
|         1789|(1789,[8,18,30,58...|
|         1922|(1922,[10,12,15,1...|
|         2226|(2226,[9,12,23,40...|
|         2237|(2237,[0,12,27,34...|
|         2351|(2351,[31,49,75,1...|
|         2542|(2542,[12,82,88,9...|
|         2610|(2610,[11,49,68,9...|
|         2709|(2709,[12,30,40,4...|
|         2727|(2727,[11,52,56,6...|
|         2943|(2943,[20,29,33,5...|
|         2974|(2974,[9,12,63,70...|
|         3681|(3681,[47,63,88,9...|
|         3797|(3797,[15,38,64,1...|
|         3942|(3942,[0,21,32,35...|
|         3956|(3956,[0,16,21,28...|
|         4260|(4260,[14,20,27,6...|
+-------------+--------------------+
only showing top 20 rows

root
 |-- Member_number: string (nullable = true)
 |-- Items: vector (nullable = true)

In [255]:
dfMembers = dfMembers.rdd.map(lambda x: 
    (x.Member_number, Vectors.dense(x.Items.toArray())) 
    ).toDF(["label","features"])
dfMembers.show()

+-----+--------------------+
|label|            features|
+-----+--------------------+
| 1249|[0.0,0.0,0.0,0.0,...|
| 1381|[0.0,1.0,0.0,0.0,...|
| 1440|[0.0,0.0,0.0,0.0,...|
| 1659|[0.0,0.0,0.0,0.0,...|
| 1789|[0.0,0.0,0.0,0.0,...|
| 1922|[0.0,0.0,0.0,0.0,...|
| 2226|[0.0,0.0,0.0,0.0,...|
| 2237|[1.0,0.0,0.0,0.0,...|
| 2351|[0.0,0.0,0.0,0.0,...|
| 2542|[0.0,0.0,0.0,0.0,...|
| 2610|[0.0,0.0,0.0,0.0,...|
| 2709|[0.0,0.0,0.0,0.0,...|
| 2727|[0.0,0.0,0.0,0.0,...|
| 2943|[0.0,0.0,0.0,0.0,...|
| 2974|[0.0,0.0,0.0,0.0,...|
| 3681|[0.0,0.0,0.0,0.0,...|
| 3797|[0.0,0.0,0.0,0.0,...|
| 3942|[1.0,0.0,0.0,0.0,...|
| 3956|[1.0,0.0,0.0,0.0,...|
| 4260|[0.0,0.0,0.0,0.0,...|
+-----+--------------------+
only showing top 20 rows



In [256]:
from pyspark.ml.clustering import KMeans, KMeansModel

kmeans = KMeans(k=5)
kmeans.setMaxIter(10)
kmeans.setWeightCol("features")

KMeans_efbc86103cec

In [257]:
# model = kmeans.fit(dfMembers)

In [258]:
# transformed = model.transform(dfMembers).select("Member_number", "Items")
# rows = transformed.collect()
# print(rows)